# Fraud Accounts (Identified): Querying and Features Extraction (Day 07)

We use this script to quering and extracting features for identified fraud accounts.

In [1]:
import pandas as pd
import numpy as np
from datetime import date

from scipy import stats
get_ipython().magic(u'config IPCompleter.greedy=True')


## Connect with the Redshift Database

In [2]:
from contextlib import closing


import psycopg2
import simplejson
import sys
reload(sys)
sys.setdefaultencoding('utf8')

DEFAULT_DB = 'data_depot'
DEFAULT_HOST = 'freshbooks-data.c8exzn6geij3.us-east-1.redshift.amazonaws.com'
DEFAULT_PORT = 5439


class PsycopgConnector:
    '''
    A database connector that uses Psycopg to connect to Redshift.

    How to play:

        psy_conn = PsycopgConnector(username, password)
        df = psy_conn.run_query(sql=sql, return_data=True)

    NOTE: This class commits queries to redshift if return_data=False.
    This means INSERT, DROP, TRUNCATE, etc. all work against the DB.
    '''

    def __init__(
        self,
        username=None,
        password=None,
        db=DEFAULT_DB,
        host=DEFAULT_HOST,
        port=DEFAULT_PORT,
    ):

        self.db = DEFAULT_DB
        self.host = DEFAULT_HOST
        self.port = DEFAULT_PORT

        self.username = username
        self.password = password

    def _get_connection(self):

        self.conn = psycopg2.connect(
            dbname=self.db,
            user=self.username,
            password=self.password,
            host=self.host,
            port=self.port
        )

        return self.conn

    def run_query(self, sql, return_data=False):

        with closing(self._get_connection()) as conn:
            with conn, conn.cursor() as cur:
                if return_data:
                    return pd.read_sql(sql=sql, con=conn)
                else:
                    cur.execute(sql)
                    

# Read the Redshift's credentials file 
with open("redshift_creds.json.nogit") as fh:
    creds = simplejson.loads(fh.read())
    
username = creds.get("user_name")
password = creds.get("password")

pig = PsycopgConnector(username, password)

In [3]:
# Testing connection
sql_test = '''SELECT * FROM report_systems LIMIT 5'''
df_test = pig.run_query(sql_test, return_data=True)
df_test

,systemid,business_id,admin_identity_id,subdomain,is_freshbooks_account_active,is_modern,most_recent_migrated_to_smux_at,is_contractor,currency_code,timezone,...,staff_count,staff_deleted_count,contractor_count,contractor_deleted_count,user_contact_count,enabled_gateway_count,google_sso_first_linked_date,google_sso_most_recent_linked_date,google_sso_first_removal_date,google_sso_most_recent_removal_date
0,595,None,None,https://sbma.freshbooks.com,1,0,None,0,USD,US/Eastern,...,2,0,0,0,0,0,None,None,None,None
1,1048,None,None,https://globacoreinc.freshbooks.com,1,0,None,0,CAD,US/Eastern,...,0,15,0,0,110,2,None,None,None,None
2,1645,None,None,https://Amfamic.freshbooks.com,1,0,None,0,USD,US/Eastern,...,0,0,0,0,0,1,None,None,None,None
3,2610,None,None,https://HRexplosion.freshbooks.com,1,0,None,0,USD,US/Eastern,...,0,0,0,0,0,0,None,None,None,None
4,2847,None,None,https://kvbtest.freshbooks.com,0,0,None,0,CAD,US/Eastern,...,0,0,0,0,0,0,None,None,None,None


## Functions

In [4]:
# Word count fuction
import re
def words_count (strg):
    
    #print(strg)
    
    if strg == '' or pd.isnull(strg):
        no_of_words = 0
        #print('NaN')
    else:
        strg_words_list = re.findall(r"[\w']+", strg)
        no_of_words = len(strg_words_list)

        
        #print(strg_words_list)
    
    return no_of_words 
    

# 1. Import Invoice Data & Extract Avg Word Counts Features

## 1.01 Invoice within 7 days

In [12]:
# SQL for impoorting all invoices created within 7 days after signup_date
sql_invoices_7days_all_accounts = '''WITH invoices_in_a_period AS (
    SELECT
            systemid,
            signup_date
    FROM report_systems rs
    JOIN ( SELECT systemid FROM data_science.fraud_accounts_final ) USING (systemid)
), invoice_created_at AS (
    SELECT
           pic.systemid,
           pic.signup_date,
           inv.invoiceid,
           inv.create_date,
           inv.description,
           inv.notes,
           inv.terms,
           inv.address,
           DATEDIFF(days, pic.signup_date, inv.created_at) AS days_to_invoice_creation
    FROM invoices_in_a_period AS pic
    LEFT JOIN coalesced_live_shards.invoice_stable as inv USING (systemid)
    WHERE ((days_to_invoice_creation BETWEEN 0 AND 7) OR days_to_invoice_creation IS NULL)
)

SELECT *
FROM invoice_created_at;'''

# Import as dataframe from redshift
# df_invoices_7days_all_accounts = pd.read_sql_query(sql_invoices_7days_all_accounts, connect_to_db)
df_invoices_7days_all_accounts = pig.run_query(sql_invoices_7days_all_accounts, return_data=True)


# Words count in invoice's description, notes, terms, address
df_invoices_7days_all_accounts['avg_wc_description_day_7'] = df_invoices_7days_all_accounts.apply(lambda x: words_count(x['description']), axis=1)
df_invoices_7days_all_accounts['avg_wc_notes_day_7'] = df_invoices_7days_all_accounts.apply(lambda x: words_count(x['notes']), axis=1)
df_invoices_7days_all_accounts['avg_wc_terms_day_7'] = df_invoices_7days_all_accounts.apply(lambda x: words_count(x['terms']), axis=1)
df_invoices_7days_all_accounts['avg_wc_address_day_7'] = df_invoices_7days_all_accounts.apply(lambda x: words_count(x['address']), axis=1)

                                                                                                                   
# Filters the text columns from the dataframe
df_invoices_7days_all_accounts_fil = df_invoices_7days_all_accounts.filter(['systemid', 
                                                                            'invoiceid', 
                                                                            'signup_date',
                                                                            'create_date', 
                                                                            'created_at',
                                                                            'days_to_invoice_creation', 
                                                                            'avg_wc_description_day_7', 
                                                                            'avg_wc_notes_day_7', 
                                                                            'avg_wc_terms_day_7',
                                                                            'avg_wc_address_day_7'])  
                                                                                                                   
# Summing (grouping) all invoices for a 'systemid'
df_word_count_7days_all_accounts_total = df_invoices_7days_all_accounts_fil.groupby('systemid').mean()  

# Final word count table
df_word_count_7days_all_accounts_final = df_word_count_7days_all_accounts_total.filter([
                                                                            'systemid',
                                                                            'signup_date',
                                                                            'avg_wc_description_day_7', 
                                                                            'avg_wc_notes_day_7', 
                                                                            'avg_wc_terms_day_7',
                                                                            'avg_wc_address_day_7'])

In [13]:
df_word_count_7days_all_accounts_final.head()

,avg_wc_description_day_7,avg_wc_notes_day_7,avg_wc_terms_day_7,avg_wc_address_day_7
systemid,,,,
140971,7.000000,18.000000,2.500000,0.0
223260,5.244009,6.211329,3.193900,0.0
280581,8.006369,0.509554,10.191083,0.0
292308,15.564935,0.359740,25.964935,0.0
537585,4.522523,2.540541,13.495495,0.0


In [14]:
df_word_count_7days_all_accounts_final.shape

(336, 4)

# 2. Report Systems Features


In [15]:
################# Import RSystems, Periodic Invoices & Client Counts Data ###############

# SQL query 
sql_rs_invoices_clients_activities_all_accounts = '''WITH periodic_report_system_activities AS (
    SELECT
        systemid,
        signup_date,
        admin_email,
        is_sales_managed,
        is_freshbooks_account_active,
        freshbooks_account_status,
        is_paying
    FROM report_systems rs
    JOIN ( SELECT systemid FROM data_science.fraud_accounts_final ) USING (systemid)
), invoice_create_date AS (
    SELECT
           pic.systemid,
           inv.invoiceid,
           inv.create_date,
           inv.created_at,
           DATEDIFF(days, pic.signup_date, inv.created_at) AS days_to_invoice_creation
    FROM periodic_report_system_activities AS pic
    LEFT JOIN coalesced_live_shards.invoice_stable as inv USING (systemid)
), invoice_grouping AS (
    SELECT
           systemid,
           COUNT(invoiceid) as invoice_count,
           SUM(CASE WHEN days_to_invoice_creation BETWEEN 0 AND 7 THEN 1 ELSE 0 END) AS invoice_count_day_7
    FROM invoice_create_date
    GROUP BY systemid
), client_crate_date AS (
     SELECT
            pic.systemid,
            usr.userid,
            usr.signup_date,
            DATEDIFF(days, pic.signup_date, usr.signup_date) AS days_to_client_creation
    FROM periodic_report_system_activities  AS pic
    LEFT JOIN coalesced_live_shards."user" as usr USING (systemid)
), client_grouping AS (
    SELECT
           systemid,
           count(userid) AS client_count,
           SUM(CASE WHEN days_to_client_creation BETWEEN 0 AND 7 THEN 1 ELSE 0 END) AS client_count_day_7

    FROM  client_crate_date
    GROUP BY systemid
)

SELECT
       systemid,
       signup_date,
       current_date as effective_date,
       DATEDIFF(days, signup_date, current_date) as days_on_platform,
       admin_email,
       is_sales_managed,
       is_freshbooks_account_active,
       is_paying,
       inv_gr.invoice_count_day_7,
       cl_gr.client_count_day_7
FROM periodic_report_system_activities 
LEFT JOIN invoice_grouping as inv_gr USING (systemid)
LEFT JOIN client_grouping AS cl_gr USING (systemid);
'''

# Import as dataframe from redshift
df_rs_invoices_clients_activities_all_accounts = pig.run_query(sql_rs_invoices_clients_activities_all_accounts, return_data=True)


In [16]:
# checking 
df_rs_invoices_clients_activities_all_accounts.tail()

,systemid,signup_date,effective_date,days_on_platform,admin_email,is_sales_managed,is_freshbooks_account_active,is_paying,invoice_count_day_7,client_count_day_7
357,771231,2013-08-16,2020-02-22,2381,thorsdalepayment@gmail.com,0,0,0,9,10
358,3022672,2017-08-17,2020-02-22,919,henrykatie043@gmail.com,0,0,0,2,3
359,2849971,2017-03-21,2020-02-22,1068,sales@reckmorph.com,0,0,0,15,17
360,1867638,2015-09-20,2020-02-22,1616,rayapenfrancishedley8@gmail.com,0,0,0,2,2
361,3726101,2018-09-27,2020-02-22,513,info@ungatecategory.com,0,0,0,3,3


In [17]:
df_rs_invoices_clients_activities_all_accounts.shape

(362, 10)

# 4. Import and Exract Features from Events Data
## 4.1 Event data collection 

In [18]:
############################### Event Features Extraction ################################

#SQL for events 
sql_events = '''WITH selected_accounts_events AS (
    SELECT systemid,
           signup_date,
           signup_datetime
    FROM report_systems
    JOIN ( SELECT systemid FROM data_science.fraud_accounts_final ) USING (systemid)
), events_activities AS (
    SELECT sae.systemid,
           signup_date,
           dd.date,
           datediff(days, signup_date, dd.date) as days_to_event,
           lower(e.event) as event,
           ec.count
    FROM selected_accounts_events AS sae
    LEFT JOIN event_counts AS ec USING (systemid)
    LEFT JOIN d_date AS dd USING (date_key)
    LEFT JOIN d_event e on ec.event_key = e.event_key
), event_groupings AS (
    SELECT distinct  ea.systemid,
                    ea.signup_date,
                    ea.date,
                    ea.event,
                    ea.count,
                    (CASE WHEN days_to_event BETWEEN 0 AND 7 THEN ea.count END) AS day_7_event
    FROM events_activities AS ea
)
SELECT systemid,
       event,
       sum(day_7_event) AS event_count_day_7
From event_groupings
GROUP BY systemid, signup_date, event
ORDER BY systemid, event_count_day_7 DESC;'''

# Import as dataframe from redshift
# df_events_all_accounts = pd.read_sql_query(sql_events, connect_to_db)
df_events_all_accounts = pig.run_query(sql_events, return_data=True)


In [19]:
# checking
df_events_all_accounts.head()

,systemid,event,event_count_day_7
0,140971,None,NaN
1,223260,logout,NaN
2,223260,delete invoice,NaN
3,223260,create payment in,NaN
4,223260,online payment received,NaN


In [20]:
df_events_all_accounts.shape

(20291, 3)

## 4.2 Removing whitespce from the event strings

In [21]:
# Removing row if there is 'None' the event cell
df_events_all_accounts = df_events_all_accounts[~df_events_all_accounts.astype(str).eq('None').any(1)]

# Replace the 'NaN' cell by zero
df_events_all_accounts.fillna(0, inplace=True)

# Using lambda function to remove the white space in the event string name
df_events_all_accounts['event_name'] = df_events_all_accounts.apply(lambda x: x['event'].replace(' ', '').replace('-','').replace('/', ''), axis=1)

In [23]:
# checking
df_events_all_accounts.head()

,systemid,event,event_count_day_7,event_name
1,223260,logout,0.0,logout
2,223260,delete invoice,0.0,deleteinvoice
3,223260,create payment in,0.0,createpaymentin
4,223260,online payment received,0.0,onlinepaymentreceived
5,223260,user survey - net-promoter-2015,0.0,usersurveynetpromoter2015


In [24]:
# list(df_events_all_accounts['event_name'])

In [25]:
# Filtered the events columns for day 7
df_events_all_accounts_day_7 = df_events_all_accounts[['systemid', 'event_count_day_7', 'event_name']]

In [26]:
df_events_all_accounts_day_7.tail()

,systemid,event_count_day_7,event_name
20286,4316674,1.0,successfulonlinepayment
20287,4316674,1.0,identitycompletedonboarding
20288,4316674,1.0,ariasupplementalplanreplaced
20289,4316674,1.0,ariatrialsubscriptioncreated
20290,4316674,1.0,ariasubscriptioncreated


In [27]:
df_events_all_accounts_day_7.shape

(20289, 3)

## 4.2 Pivote the events (each unique event become a column)

In [28]:
### Pivote the Day 7 Events (Each Unique Event Become a Column)###

# Pivot table based on the unique column value in 'event_name'
df_events_all_accounts_day_7 = df_events_all_accounts_day_7.pivot_table(values='event_count_day_7', columns='event_name', index='systemid', aggfunc=np.sum,  fill_value=0)

# Drop the old column name
df_events_all_accounts_day_7.columns.name = None

# Reset the index
df_events_all_accounts_day_7 = df_events_all_accounts_day_7.reset_index()

# Replace 'NaN' with zero
df_events_all_accounts_day_7.fillna(0, inplace=True)

In [29]:
# checking
df_events_all_accounts_day_7.tail()

,systemid,acceptestimate,accesstokencreated,accountmonthiversary,activateclient,activateestimate,activateexpense,activateinvoice,activateitem,activatepayment,...,usersurveynetpromoter2018,usersurveynetpromoter2019,usersurveysubmitted,verifycallback,verifymigration,viewedcreupgradepage,viewestimate,viewinvoice,welcomeaccount,zendesksupportemail
355,4202030,0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,13,1,0
356,4238602,0,5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,39,1,0
357,4244390,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
358,4296696,0,11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,6,1,0
359,4316674,0,22,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,11,1,0


In [30]:
df_events_all_accounts_day_7.shape

(360, 476)

# 5. Merging all data: Report system, average word count and event data

In [31]:
# Merging report system and events data for day 7 period
df_rs_events_day_7 = pd.merge(df_rs_invoices_clients_activities_all_accounts, df_events_all_accounts_day_7,
                             on='systemid', how='left')

# Merging average word count with 'df_rs_events_day_7'
df_rs_events_avg_wc_day_7 = pd.merge(df_rs_events_day_7, df_word_count_7days_all_accounts_final,
                                    on='systemid', how='left')

In [32]:
df_rs_events_avg_wc_day_7.head()

,systemid,signup_date,effective_date,days_on_platform,admin_email,is_sales_managed,is_freshbooks_account_active,is_paying,invoice_count_day_7,client_count_day_7,...,verifymigration,viewedcreupgradepage,viewestimate,viewinvoice,welcomeaccount,zendesksupportemail,avg_wc_description_day_7,avg_wc_notes_day_7,avg_wc_terms_day_7,avg_wc_address_day_7
0,2515764,2016-07-14,2020-02-22,1318,sales@technicalsupporting.com,0,0,0,10,9,...,0.0,3.0,0.0,10.0,0.0,0.0,3.300000,1.000000,2.30000,0.0
1,790342,2013-09-09,2020-02-22,2357,justin@freshlemonsolutions.com,0,0,0,5,4,...,0.0,0.0,0.0,0.0,0.0,0.0,5.400000,4.200000,46.20000,0.0
2,2336324,2016-04-20,2020-02-22,1403,arossrcr@roughcountryrusticfurniture.com,0,0,0,33,31,...,0.0,0.0,0.0,76.0,0.0,0.0,3.848485,4.636364,6.00000,0.0
3,1171895,2014-08-28,2020-02-22,2004,nazia.quraishi@technicianforyou.com,0,0,0,12,13,...,0.0,0.0,0.0,0.0,0.0,0.0,1.750000,3.250000,5.25000,0.0
4,2550103,2016-08-05,2020-02-22,1296,americanguy009@gmail.com,0,0,0,82,80,...,0.0,0.0,0.0,106.0,0.0,0.0,4.682927,0.000000,180.54878,0.0


In [ ]:
# df_rs_events_avg_wc_day_7['signup_date']

# 6. Filtering out FreshBooks test accounts 

In [36]:
################# Filtering Out FreshBooks Test Accounts #############################################################

# Import Freshbooks test accounts email from CSV file (non-freshbooks email)
fb_test_emails = pd.read_csv(
    "/Users/dwahid/Documents/GitHub/fraud_detection/data/freshbooks_test_email/non-fb-testing-emails.tsv", sep="\t")
fb_test_email_list = list(fb_test_emails['email'])

In [ ]:
# fb_test_email_list

In [37]:
# Function: Filtering FB test account by using admin email
from difflib import SequenceMatcher

def email_match(em, email_list):
    
    L = len(email_list)
#     print('L', L)
#     print('em-before-loop: ', em)
    match_score = 0
#     x = float(em)
    
    for i in range(0, L):
#         if math.isnan(x):
#             match_score = 0
#             break;
        if pd.isnull(em):
            match_score = 0
            break;
        else: 
            match_score =  max(match_score, SequenceMatcher(None,em, email_list[i]).ratio())
#             print(i, em, email_list[i], match_score)

    return match_score
    


In [38]:
# Filtering final data from the FreshBooks Test emails
df_rs_events_avg_wc_day_7_noFBtest = df_rs_events_avg_wc_day_7[
    df_rs_events_avg_wc_day_7.apply(lambda x: email_match(x['admin_email'], fb_test_email_list) < 0.95, axis=1)]

In [40]:
df_rs_events_avg_wc_day_7_noFBtest['signup_date'].head()

0    2016-07-14
1    2013-09-09
2    2016-04-20
3    2014-08-28
4    2016-08-05
Name: signup_date, dtype: object

In [ ]:
# list(df_rs_events_avg_wc_day_7_noFBtest)

# 7. Filtering only important features: Day 7

In [41]:
# Importing importing features list
important_features = pd.read_csv( 
    "/Users/dwahid/Documents/GitHub/fraud_detection/src/important_features/important_features_day_07_new_accounts.tsv", sep="\n,")

# Get the important feature as a list
imp_features_list = list(important_features['important_feature'])

In [42]:
len(imp_features_list)

93

In [43]:
# Adding missing important feature column with zero values (if there any!)
for i in range(len(imp_features_list)):
    if imp_features_list[i] in df_rs_events_avg_wc_day_7_noFBtest.columns:
#         print("True")
        continue;
        
    else:
        print("False: ", imp_features_list[i])
        df_rs_events_avg_wc_day_7_noFBtest[imp_features_list[i]] = 0


In [44]:
df_rs_events_avg_wc_day_7_noFBtest.shape

(362, 496)

In [ ]:
# list(df_rs_events_avg_wc_day_7_noFBtest)

In [ ]:
# df_rs_events_avg_wc_day_7_noFBtest['signup_date']

In [45]:
# Filtering only important features 
df_imp_features_new_accounts_day_7 =\
            df_rs_events_avg_wc_day_7_noFBtest[df_rs_events_avg_wc_day_7_noFBtest.columns.intersection(imp_features_list)]

In [46]:
df_imp_features_new_accounts_day_7.head()

,systemid,signup_date,effective_date,days_on_platform,admin_email,is_sales_managed,is_freshbooks_account_active,is_paying,invoice_count_day_7,client_count_day_7,...,avg_wc_notes_day_7,avg_wc_terms_day_7,avg_wc_address_day_7,activateotherincome,archiveotherincome,archivetask,bulkimportclientscomplete,createbanktransfer,deleteuser,updateservice
0,2515764,2016-07-14,2020-02-22,1318,sales@technicalsupporting.com,0,0,0,10,9,...,1.000000,2.30000,0.0,0,0,0,0,0,0,0
1,790342,2013-09-09,2020-02-22,2357,justin@freshlemonsolutions.com,0,0,0,5,4,...,4.200000,46.20000,0.0,0,0,0,0,0,0,0
2,2336324,2016-04-20,2020-02-22,1403,arossrcr@roughcountryrusticfurniture.com,0,0,0,33,31,...,4.636364,6.00000,0.0,0,0,0,0,0,0,0
3,1171895,2014-08-28,2020-02-22,2004,nazia.quraishi@technicianforyou.com,0,0,0,12,13,...,3.250000,5.25000,0.0,0,0,0,0,0,0,0
4,2550103,2016-08-05,2020-02-22,1296,americanguy009@gmail.com,0,0,0,82,80,...,0.000000,180.54878,0.0,0,0,0,0,0,0,0


In [47]:
df_imp_features_new_accounts_day_7 = df_imp_features_new_accounts_day_7.reindex(
    sorted(df_imp_features_new_accounts_day_7.columns), axis=1)

In [ ]:
# list(df_final_features_new_accounts_day_7)

In [ ]:
# df_imp_features_new_accounts_day_7['signup_date']

In [48]:
# Drop rows with nan value
df_imp_features_new_accounts_day_7 = df_imp_features_new_accounts_day_7.dropna()

# 8. Filtering inactive users' accounts

In [49]:
ex_cols_list = ['admin_email','days_on_platform', 'effective_date', 'signup_date', 'systemid']
cols_list = list(df_imp_features_new_accounts_day_7) 
cols = list(set(cols_list) - set(ex_cols_list))

In [ ]:
# cols

In [50]:
# Function for aggregating selected column values
def cell_value_sum (row, cols):
    #print(row)
    sum = 0
    for i in cols:
        #print(i)
        #print(i, row[i])
        sum = sum + row[i]
    
    #print('Final sum: ', sum)
    return sum


In [51]:
# Fltering out all inactive users accounts
df_final_features_new_accounts_day_7 =\
        df_imp_features_new_accounts_day_7[df_imp_features_new_accounts_day_7.apply(lambda x: cell_value_sum(x, cols) > 0, axis=1)]

In [52]:
df_final_features_new_accounts_day_7.head()

,activateexpense,activateotherincome,activatepayment,admin_email,admindeactivation,adminonlinepaymentattempt,adminpayinvoiceonlineinvoice,adminpayinvoiceonlinelistview,archiveclient,archiveexpense,...,updatecategory,updateclient,updatecompanyprofile,updatecontractor,updatecreditnote,updateestimate,updateexpense,updateinvoicesample,updateitem,updateservice
0,0.0,0,0.0,sales@technicalsupporting.com,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0,0.0,justin@freshlemonsolutions.com,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0,0.0,arossrcr@roughcountryrusticfurniture.com,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0,0.0,nazia.quraishi@technicianforyou.com,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0,0.0,americanguy009@gmail.com,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [ ]:
# df_final_features_new_accounts_day_7['signup_date']

# 9. Saving the filtered features data for new accounts

In [53]:
# Export filtered features data fro new accounts
today = str(date.today())
path = "/Users/dwahid/Documents/GitHub/fraud_detection/data/analyzing_fraud_accounts/fraud_accounts_features_day_07.tsv"
df_final_features_new_accounts_day_7.to_csv(path, sep="\t", index=False)

In [54]:
list(df_final_features_new_accounts_day_7)

['activateexpense',
 'activateotherincome',
 'activatepayment',
 'admin_email',
 'admindeactivation',
 'adminonlinepaymentattempt',
 'adminpayinvoiceonlineinvoice',
 'adminpayinvoiceonlinelistview',
 'archiveclient',
 'archiveexpense',
 'archiveotherincome',
 'archiveproject',
 'archivetask',
 'autobillpayment',
 'avg_wc_address_day_7',
 'avg_wc_description_day_7',
 'avg_wc_notes_day_7',
 'avg_wc_terms_day_7',
 'bulkimportclientscomplete',
 'client_count_day_7',
 'clientimportcsvsucceeded',
 'clientlimitupgradenudge',
 'createbankaccount',
 'createbanktransaction',
 'createbanktransfer',
 'createcategory',
 'createcontact',
 'createcontractor',
 'createcreditnote',
 'createdexpense',
 'createestimate',
 'createexpense',
 'createitem',
 'createotherincome',
 'createreceipt',
 'createservice',
 'creditcardclientaccessgranted',
 'customemailsignature',
 'days_on_platform',
 'declinedonlinepaymentnotification',
 'deletebusinesspartner',
 'deletecollaborator',
 'deletecreditnote',
 'deletee

In [55]:
df_final_features_new_accounts_day_7.shape

(334, 93)